# Pandas - Pokročilé transformace dat

## Úvod do pokročilých transformací

Tento notebook ukazuje pokročilé transformace dat s vizualizací "před → po" pro lepší pochopení.

### Wide vs Long formát

```
WIDE FORMÁT (široký) - intuitivní pro čtení
┌─────────┬───────┬──────┬────────┐
│  mesto  │ leden │ unor │ brezen │
├─────────┼───────┼──────┼────────┤
│  praha  │   -2  │   1  │    8   │
│   brno  │   -1  │   2  │    9   │
└─────────┴───────┴──────┴────────┘
         ↕ TRANSFORMACE ↕
LONG FORMÁT (dlouhý) - lepší pro analýzy  
┌─────────┬────────┬─────────┐
│  mesto  │ mesic  │ teplota │
├─────────┼────────┼─────────┤
│  praha  │ leden  │   -2    │
│  praha  │ unor   │    1    │
│  praha  │ brezen │    8    │
│   brno  │ leden  │   -1    │
│   brno  │ unor   │    2    │
│   brno  │ brezen │    9    │
└─────────┴────────┴─────────┘
```

#### Wide formát (široký)
- Každá proměnná má vlastní sloupec
- Intuitivní pro lidské čtení
- Kompaktnější zobrazení

#### Long formát (dlouhý)  
- Proměnné jsou ve sloupcích "variable" a "value"
- Lepší pro analýzy a vizualizace
- Snadno se filtruje a grupuje

## Přehled transformačních metod

### Metody pro změnu struktury dat
| Metoda | Co ovlivňuje | Popis | Příklad použití |
|--------|--------------|-------|-----------------|
| `df.stack()` | **Sloupce → Index** | Převod sloupců na řádky (wide → long) | `df.stack()` |
| `df.unstack()` | **Index → Sloupce** | Převod řádků na sloupce (long → wide) | `df.unstack()` |
| `pd.melt()` | **Sloupce → Řádky** | Flexibilní wide → long transformace | `pd.melt(df, id_vars, value_vars)` |
| `series.reset_index()` | **Index → Sloupce** | Převod indexu na sloupec | `series.reset_index()` |
| `df.set_index()` | **Sloupce → Index** | Nastavení sloupce jako index | `df.set_index('col')` |

### Metody pro rozbalování a analýzu
| Metoda | Co ovlivňuje | Popis | Příklad použití |
|--------|--------------|-------|-----------------|
| `df.explode()` | **Řádky** | Rozbalení seznamů do řádků | `df.explode('column')` |
| `pd.crosstab()` | **Celá struktura** | Křížové tabulky četností | `pd.crosstab(index, columns)` |
| `pd.MultiIndex.from_tuples()` | **Index** | Vytvoření MultiIndexu | `pd.MultiIndex.from_tuples(tuples)` |

### Klíčové parametry
| Parametr | Metoda | Popis | Výchozí hodnota |
|----------|--------|-------|-----------------|
| `level` | stack/unstack | Úroveň indexu k transformaci | `-1` (poslední) |
| `id_vars` | melt | Sloupce k zachování jako identifikátory | `None` |
| `value_vars` | melt | Sloupce k transformaci | `None` (všechny) |
| `var_name` | melt | Název nového sloupce proměnných | `'variable'` |
| `value_name` | melt | Název nového sloupce hodnot | `'value'` |
| `normalize` | crosstab | Normalizace hodnot (True/False/'index'/'columns') | `False` |
| `margins` | crosstab | Přidání součtů | `False` |
| `margins_name` | crosstab | Název pro součtový řádek/sloupec | `'All'` |
| `drop` | reset_index | Zrušení původního indexu | `False` |

### Další použité metody
| Metoda | Popis | Příklad |
|--------|-------|---------|
| `df.copy()` | Vytvoření kopie DataFrame | `df.copy()` |
| `df.iloc[]` | Přístup k datům podle pozice | `df.iloc[:, 0]` |
| `df.columns` | Seznam názvů sloupců | `df.columns[0]` |
| `series.name` | Název Series | `series.name = 'teplota'` |
| `df.round()` | Zaokrouhlení hodnot | `df.round(3)` |

## Stack - Sloupce do indexu

**Co dělá:** Bere sloupce a převádí je na další úroveň indexu (sloupce → řádky)

### Vizualizace stack transformace

```
PŘED - Wide DataFrame
┌─────────┬───────┬──────┬────────┐
│  index  │ leden │ unor │ brezen │
├─────────┼───────┼──────┼────────┤
│  praha  │   -2  │   1  │    8   │
│   brno  │   -1  │   2  │    9   │
└─────────┴───────┴──────┴────────┘
                  ↓ .stack()
PO - Long Series s MultiIndex
┌─────────┬────────┬─────────┐
│ uroven 0│ uroven 1│ hodnota │
├─────────┼────────┼─────────┤
│  praha  │ leden  │   -2    │
│  praha  │ unor   │    1    │
│  praha  │ brezen │    8    │
│   brno  │ leden  │   -1    │
│   brno  │ unor   │    2    │
│   brno  │ brezen │    9    │
└─────────┴────────┴─────────┘
```

### Praktický příklad

In [1]:
import pandas as pd
import numpy as np

# Teploty ve wide formátu
teploty_wide = pd.DataFrame({
    'leden': [-2, -1],
    'unor': [1, 2], 
    'brezen': [8, 9]
}, index=['praha', 'brno'])

print("PŘED - Wide formát:")
print(teploty_wide)
print()

# Stack - sloupce do indexu
teploty_long = teploty_wide.stack()
print("PO - Long formát:")
print(teploty_long)
print()
print("Typ výsledku:", type(teploty_long))
print("Název Series:", teploty_long.name if teploty_long.name else "bez názvu")

PŘED - Wide formát:
       leden  unor  brezen
praha     -2     1       8
brno      -1     2       9

PO - Long formát:
praha  leden    -2
       unor      1
       brezen    8
brno   leden    -1
       unor      2
       brezen    9
dtype: int64

Typ výsledku: <class 'pandas.core.series.Series'>
Název Series: bez názvu


### Stack s pojmenováním

In [2]:
import pandas as pd

# Pojmenování výsledné Series
teploty_pojmenovane = teploty_wide.stack()
teploty_pojmenovane.name = 'teplota'

print("Stack s pojmenováním:")
print(teploty_pojmenovane)
print()

# Převod zpět na DataFrame
teploty_df = teploty_pojmenovane.reset_index()
teploty_df.columns = ['mesto', 'mesic', 'teplota']
print("Jako DataFrame:")
print(teploty_df)

Stack s pojmenováním:
praha  leden    -2
       unor      1
       brezen    8
brno   leden    -1
       unor      2
       brezen    9
Name: teplota, dtype: int64

Jako DataFrame:
   mesto   mesic  teplota
0  praha   leden       -2
1  praha    unor        1
2  praha  brezen        8
3   brno   leden       -1
4   brno    unor        2
5   brno  brezen        9


## Unstack - Index do sloupců

**Co dělá:** Bere úroveň indexu a převádí ji na sloupce (řádky → sloupce)

### Vizualizace unstack transformace

```
PŘED - Long Series s MultiIndex
┌─────────┬────────┬─────────┐
│ uroven 0│ uroven 1│ hodnota │
├─────────┼────────┼─────────┤
│  praha  │ leden  │   -2    │
│  praha  │ unor   │    1    │
│  praha  │ brezen │    8    │
│   brno  │ leden  │   -1    │
│   brno  │ unor   │    2    │
│   brno  │ brezen │    9    │
└─────────┴────────┴─────────┘
                  ↓ .unstack()
PO - Wide DataFrame
┌─────────┬───────┬──────┬────────┐
│  index  │ leden │ unor │ brezen │
├─────────┼───────┼──────┼────────┤
│  praha  │   -2  │   1  │    8   │
│   brno  │   -1  │   2  │    9   │
└─────────┴───────┴──────┴────────┘
```

### Praktický příklad

In [3]:
import pandas as pd

# Začneme s long formátem (MultiIndex Series)
mesta_mesice = pd.MultiIndex.from_tuples([
    ('praha', 'leden'), ('praha', 'unor'), ('praha', 'brezen'),
    ('brno', 'leden'), ('brno', 'unor'), ('brno', 'brezen')
], names=['mesto', 'mesic'])

teploty_series = pd.Series([-2, 1, 8, -1, 2, 9], index=mesta_mesice, name='teplota')

print("PŘED - Long formát (Series s MultiIndex):")
print(teploty_series)
print()

# Unstack - poslední úroveň indexu do sloupců
teploty_unstacked = teploty_series.unstack()
print("PO - Wide formát:")
print(teploty_unstacked)
print()

# Unstack konkrétní úrovně - použijeme číslo sloupce
teploty_unstacked_level0 = teploty_series.unstack(level=0)
print("Unstack úrovně 0 (město):")
print(teploty_unstacked_level0)

PŘED - Long formát (Series s MultiIndex):
mesto  mesic 
praha  leden    -2
       unor      1
       brezen    8
brno   leden    -1
       unor      2
       brezen    9
Name: teplota, dtype: int64

PO - Wide formát:
mesic  brezen  leden  unor
mesto                     
brno        9     -1     2
praha       8     -2     1

Unstack úrovně 0 (město):
mesto   brno  praha
mesic              
brezen     9      8
leden     -1     -2
unor       2      1


### Práce s chybějícími hodnotami

In [4]:
import pandas as pd

# Data s chybějícími hodnotami
neuplna_data = pd.DataFrame({
    'leden': [-2, -1, np.nan],
    'unor': [1, np.nan, 3],
    'brezen': [8, 9, 10]
}, index=['praha', 'brno', 'ostrava'])

print("Data s NaN hodnotami:")
print(neuplna_data)
print()

# Stack ignoruje NaN automaticky
stacked_s_nan = neuplna_data.stack()
print("Po stack (NaN zmizí):")
print(stacked_s_nan)
print()

# Unstack vrátí NaN tam, kde chybí data
unstacked_zpet = stacked_s_nan.unstack()
print("Po unstack zpět:")
print(unstacked_zpet)

Data s NaN hodnotami:
         leden  unor  brezen
praha     -2.0   1.0       8
brno      -1.0   NaN       9
ostrava    NaN   3.0      10

Po stack (NaN zmizí):
praha    leden     -2.0
         unor       1.0
         brezen     8.0
brno     leden     -1.0
         brezen     9.0
ostrava  unor       3.0
         brezen    10.0
dtype: float64

Po unstack zpět:
         leden  unor  brezen
praha     -2.0   1.0     8.0
brno      -1.0   NaN     9.0
ostrava    NaN   3.0    10.0


## Melt - Flexibilní wide → long

**Co dělá:** Přetavuje sloupce na řádky, ale flexibilněji než stack - můžete si vybrat které sloupce transformovat

### Vizualizace melt transformace

```
PŘED - Wide DataFrame s ID sloupci
┌─────────┬───────┬──────┬────────┐
│  mesto  │ leden │ unor │ brezen │
├─────────┼───────┼──────┼────────┤
│  praha  │   -2  │   1  │    8   │
│   brno  │   -1  │   2  │    9   │
└─────────┴───────┴──────┴────────┘
                  ↓ pd.melt()
PO - Long DataFrame s variable/value sloupci
┌─────────┬────────┬─────────┐
│  mesto  │ mesic  │ teplota │ 
├─────────┼────────┼─────────┤
│  praha  │ leden  │   -2    │
│   brno  │ leden  │   -1    │
│  praha  │ unor   │    1    │
│   brno  │ unor   │    2    │
│  praha  │ brezen │    8    │
│   brno  │ brezen │    9    │
└─────────┴────────┴─────────┘
```

### Základní použití

In [5]:
import pandas as pd

# Teploty s městem jako sloupcem
meteo_wide = pd.DataFrame({
    'mesto': ['praha', 'brno'],
    'leden': [-2, -1],
    'unor': [1, 2],
    'brezen': [8, 9]
})

print("PŘED - Wide formát:")
print(meteo_wide)
print()

# Základní melt - použijeme sloupec 0 jako id_vars
meteo_melted = pd.melt(meteo_wide, 
                      id_vars=[meteo_wide.columns[0]],  # sloupec 0 = 'mesto'
                      var_name='mesic',                 # název pro proměnné
                      value_name='teplota')             # název pro hodnoty

print("PO - Long formát:")
print(meteo_melted)

PŘED - Wide formát:
   mesto  leden  unor  brezen
0  praha     -2     1       8
1   brno     -1     2       9

PO - Long formát:
   mesto   mesic  teplota
0  praha   leden       -2
1   brno   leden       -1
2  praha    unor        1
3   brno    unor        2
4  praha  brezen        8
5   brno  brezen        9


### Melt s výběrem sloupců

In [6]:
import pandas as pd

# Více dat pro ukázku
rozsirena_data = pd.DataFrame({
    'mesto': ['praha', 'brno'],
    'leden_teplota': [-2, -1],
    'unor_teplota': [1, 2],
    'leden_srazky': [45, 50],
    'unor_srazky': [38, 42],
    'nadmorska_vyska': [399, 237]
})

print("Rozšířená data:")
print(rozsirena_data)
print()

# Melt pouze teplotních sloupců - použijeme čísla sloupců
pouze_teploty = pd.melt(rozsirena_data,
                       id_vars=[rozsirena_data.columns[0], rozsirena_data.columns[5]], # sloupce 0,5
                       value_vars=[rozsirena_data.columns[1], rozsirena_data.columns[2]], # sloupce 1,2
                       var_name='mesic_typ',
                       value_name='hodnota')

print("Pouze teploty:")
print(pouze_teploty)

Rozšířená data:
   mesto  leden_teplota  unor_teplota  leden_srazky  unor_srazky  \
0  praha             -2             1            45           38   
1   brno             -1             2            50           42   

   nadmorska_vyska  
0              399  
1              237  

Pouze teploty:
   mesto  nadmorska_vyska      mesic_typ  hodnota
0  praha              399  leden_teplota       -2
1   brno              237  leden_teplota       -1
2  praha              399   unor_teplota        1
3   brno              237   unor_teplota        2


## Explode - Rozbalování seznamů

**Co dělá:** Rozbaluje seznamy v buňkách do samostatných řádků

### Vizualizace explode transformace

```
PŘED - DataFrame se seznamy v buňkách
┌─────────┬─────────────────┐
│  mesic  │     mereni      │
├─────────┼─────────────────┤
│  leden  │  [15, 18, 20]   │
│   unor  │  [22, 25, 28]   │
└─────────┴─────────────────┘
           ↓ .explode()
PO - Rozbalené řádky (duplicitní indexy)
┌─────────┬─────────┐
│  mesic  │ mereni  │
├─────────┼─────────┤
│  leden  │   15    │ ← index 0
│  leden  │   18    │ ← index 0  
│  leden  │   20    │ ← index 0
│   unor  │   22    │ ← index 1
│   unor  │   25    │ ← index 1
│   unor  │   28    │ ← index 1
└─────────┴─────────┘
```

### Praktický příklad

In [7]:
# Data se seznamy měření
data_se_seznamy = pd.DataFrame({
    'mesic': ['leden', 'unor'],
    'mereni_teploty': [[15, 18, 20], [22, 25, 28]],
    'pocet_dni': [31, 28]
})

print("PŘED - Data se seznamy:")
print(data_se_seznamy)
print()

# Explode seznamů - použijeme číslo sloupce
exploded_data = data_se_seznamy.explode(data_se_seznamy.columns[1])  # sloupec 1
print("PO - Rozbalené seznamy:")
print(exploded_data)
print()

# Reset indexu pro lepší přehlednost
exploded_clean = exploded_data.reset_index(drop=True)
print("S resetovaným indexem:")
print(exploded_clean)

PŘED - Data se seznamy:
   mesic mereni_teploty  pocet_dni
0  leden   [15, 18, 20]         31
1   unor   [22, 25, 28]         28

PO - Rozbalené seznamy:
   mesic mereni_teploty  pocet_dni
0  leden             15         31
0  leden             18         31
0  leden             20         31
1   unor             22         28
1   unor             25         28
1   unor             28         28

S resetovaným indexem:
   mesic mereni_teploty  pocet_dni
0  leden             15         31
1  leden             18         31
2  leden             20         31
3   unor             22         28
4   unor             25         28
5   unor             28         28


### Explode více sloupců najednou

In [8]:
# Data s více seznamy
vice_seznamu = pd.DataFrame({
    'den': ['pond', 'uter'],
    'teploty': [[10, 15], [12, 18]], 
    'srazky': [[2, 0], [5, 3]],
    'lokace': ['praha', 'brno']
})

print("Data s více seznamy:")
print(vice_seznamu)
print()

# Explode více sloupců současně (Pandas 1.3+) - použijeme čísla sloupců
try:
    exploded_multiple = vice_seznamu.explode([vice_seznamu.columns[1], vice_seznamu.columns[2]])  # sloupce 1,2
    print("Explode více sloupců:")
    print(exploded_multiple)
except TypeError:
    # Pro starší verze Pandas
    print("Pro starší Pandas - postupně:")
    temp_exploded = vice_seznamu.explode(vice_seznamu.columns[1])  # sloupec 1
    fully_exploded = temp_exploded.explode(vice_seznamu.columns[2])  # sloupec 2
    print(fully_exploded)

Data s více seznamy:
    den   teploty  srazky lokace
0  pond  [10, 15]  [2, 0]  praha
1  uter  [12, 18]  [5, 3]   brno

Explode více sloupců:
    den teploty srazky lokace
0  pond      10      2  praha
0  pond      15      0  praha
1  uter      12      5   brno
1  uter      18      3   brno


## Crosstab - Křížové tabulky

**Co dělá:** Vytváří křížové tabulky četností mezi dvěma nebo více proměnnými

### Vizualizace crosstab transformace

```
PŘED - Surová kategorická data (long formát)
┌─────────┬─────────┬─────────────────┐
│  mesto  │  mesic  │ kategorie_teplo │
├─────────┼─────────┼─────────────────┤
│  praha  │  leden  │     studeno     │
│  praha  │   unor  │     studeno     │
│   brno  │  leden  │     studeno     │
│   brno  │   unor  │      teplo      │
│  praha  │ brezen  │      teplo      │
│   brno  │ brezen  │      teplo      │
└─────────┴─────────┴─────────────────┘
               ↓ pd.crosstab()
PO - Křížová tabulka (agregované četnosti)
┌─────────┬─────────┬───────┐
│  mesto  │ studeno │ teplo │
├─────────┼─────────┼───────┤
│   brno  │    1    │   2   │
│  praha  │    2    │   1   │
└─────────┴─────────┴───────┘
```

### Základní křížová tabulka

In [9]:
# Kategorická data pro crosstab
meteo_kategorie = pd.DataFrame({
    'mesto': ['praha', 'praha', 'brno', 'brno', 'praha', 'brno'],
    'mesic': ['leden', 'unor', 'leden', 'unor', 'brezen', 'brezen'],
    'kategorie_teploty': ['studeno', 'studeno', 'studeno', 'teplo', 'teplo', 'teplo']
})

print("PŘED - Surová kategorická data:")
print(meteo_kategorie)
print()

# Základní crosstab - použijeme čísla sloupců
krizova_tabulka = pd.crosstab(meteo_kategorie.iloc[:, 0],  # sloupec 0 = mesto
                             meteo_kategorie.iloc[:, 2])   # sloupec 2 = kategorie_teploty

print("PO - Křížová tabulka:")
print(krizova_tabulka)

PŘED - Surová kategorická data:
   mesto   mesic kategorie_teploty
0  praha   leden           studeno
1  praha    unor           studeno
2   brno   leden           studeno
3   brno    unor             teplo
4  praha  brezen             teplo
5   brno  brezen             teplo

PO - Křížová tabulka:
kategorie_teploty  studeno  teplo
mesto                            
brno                     1      2
praha                    2      1


### Crosstab s marginálními součty

In [10]:
# S přidáním součtů - použijeme čísla sloupců
krizova_s_marginy = pd.crosstab(meteo_kategorie.iloc[:, 0],  # sloupec 0
                               meteo_kategorie.iloc[:, 2],   # sloupec 2
                               margins=True,                 # přidat součty
                               margins_name='Celkem')        # název pro součty

print("Křížová tabulka s marginálními součty:")
print(krizova_s_marginy)
print()

# Normalizace na procenta
krizova_procenta = pd.crosstab(meteo_kategorie.iloc[:, 0],  # sloupec 0
                              meteo_kategorie.iloc[:, 2],   # sloupec 2
                              normalize='index')            # procenta podle řádků

print("Normalizace podle řádků (procenta):")
print(krizova_procenta.round(3))

Křížová tabulka s marginálními součty:
kategorie_teploty  studeno  teplo  Celkem
mesto                                    
brno                     1      2       3
praha                    2      1       3
Celkem                   3      3       6

Normalizace podle řádků (procenta):
kategorie_teploty  studeno  teplo
mesto                            
brno                 0.333  0.667
praha                0.667  0.333


### Vícerozměrná křížová tabulka

In [11]:
# Přidání třetí dimenze
rozsirenа_data = meteo_kategorie.copy()
rozsirenа_data['rocni_doba'] = ['zima', 'zima', 'zima', 'zima', 'jaro', 'jaro']

print("Rozšířená data o roční dobu:")
print(rozsirenа_data)
print()

# Vícerozměrná křížová tabulka - použijeme čísla sloupců
vicerozm_tabulka = pd.crosstab([rozsirenа_data.iloc[:, 0], rozsirenа_data.iloc[:, 3]],  # sloupce 0,3
                              rozsirenа_data.iloc[:, 2])                                # sloupec 2

print("Vícerozměrná křížová tabulka:")
print(vicerozm_tabulka)

Rozšířená data o roční dobu:
   mesto   mesic kategorie_teploty rocni_doba
0  praha   leden           studeno       zima
1  praha    unor           studeno       zima
2   brno   leden           studeno       zima
3   brno    unor             teplo       zima
4  praha  brezen             teplo       jaro
5   brno  brezen             teplo       jaro

Vícerozměrná křížová tabulka:
kategorie_teploty  studeno  teplo
mesto rocni_doba                 
brno  jaro               0      1
      zima               1      1
praha jaro               0      1
      zima               2      0


## Praktické tipy

### Kdy použít kterou metodu

| Situace | Doporučená metoda | Důvod |
|---------|------------------|-------|
| Jednoduché převrácení sloupců/řádků | `stack()` / `unstack()` | Rychlé a přímočaré |
| Potřebuji zachovat některé sloupce | `melt()` | Flexibilní výběr sloupců |
| Mám seznamy v buňkách | `explode()` | Jednoduché rozbalení |
| Chci analýzu četností | `crosstab()` | Přímý výstup křížové tabulky |

### Částé chyby a řešení

In [12]:
# Chyba: Zapomenutí na MultiIndex po stack()
data_wide = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
stacked = data_wide.stack()

print("Stack vytvoří Series s MultiIndex:")
print(f"Typ: {type(stacked)}")
print(f"Index: {stacked.index}")
print()

# Řešení: Reset indexu pro DataFrame
stacked_df = stacked.reset_index()
stacked_df.columns = ['radek', 'sloupec', 'hodnota']
print("Po reset_index:")
print(stacked_df)

Stack vytvoří Series s MultiIndex:
Typ: <class 'pandas.core.series.Series'>
Index: MultiIndex([(0, 'a'),
            (0, 'b'),
            (1, 'a'),
            (1, 'b')],
           )

Po reset_index:
   radek sloupec  hodnota
0      0       a        1
1      0       b        3
2      1       a        2
3      1       b        4


## Shrnutí

### Klíčové principy transformací:

- **Stack/Unstack** pracují s indexy - nejrychlejší pro jednoduché případy
- **Melt** pracuje se sloupci - nejflexibilnější pro výběr dat
- **Explode** rozbaluje seznamy - jedinečné pro vnořené struktury  
- **Crosstab** vytváří novou strukturu - zaměřené na křížové analýzy

### Obecný workflow:
1. Určete výchozí a cílovou strukturu dat
2. Vyberte nejvhodnější metodu podle situace
3. Otestujte na malém vzorku dat
4. Ošetřete chybějící hodnoty a názvy sloupců